In [133]:
import numpy as np
import pandas as pd
import math
from pathlib import Path
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [5]:
library_root = Path('../Dataset')
excel_name = 'EASiRA data extract_061622.xlsx'
excel = pd.ExcelFile(library_root / excel_name)
df = pd.read_excel(excel, 'EASiRA_DATA_LABELS_2022-06-16_1')

In [10]:
df.head(5)

Record ID Event Name   Study ID  Gender  Age (as of today)  \
0   EARA-01    Visit 1    EARA-01  Female                NaN   
1   EARA-01    Visit 3        NaN     NaN                NaN   
2   EARA-02    Visit 1          2  Female               54.0   
3   EARA-03    Visit 1  EARA03-V1    Male               73.0   
4   EARA-03    Visit 3        NaN     NaN                NaN   

  Race (choice=White/Caucasian) Race (choice=Black/African-American)  \
0                       Checked                            Unchecked   
1                           NaN                                  NaN   
2                       Checked                            Unchecked   
3                       Checked                            Unchecked   
4                           NaN                                  NaN   

  Race (choice=Afro-Caribbean) Race (choice=Native American/Alaska Native)  \
0                    Unchecked                                   Unchecked   
1                          NaN                                         NaN   
2                    Unchecked                                   Unchecked   
3                    Unchecked                                   Unchecked   
4                          NaN                                         NaN   

  Race (choice=Asian) Race (choice=Pacific Islander)  \
0           Unchecked                      Unchecked   
1                 NaN                            NaN   
2           Unchecked                      Unchecked   
3           Unchecked                      Unchecked   
4                 NaN                            NaN   

  Race (choice=Other (specify)) Race (choice=Unknown)  \
0                     Unchecked             Unchecked   
1                           NaN                   NaN   
2                     Unchecked             Unchecked   
3                     Unchecked             Unchecked   
4                           NaN                   NaN   

  Race (choice=NOT REPORTED)  If other race, please specify  \
0                  Unchecked                            NaN   
1                        NaN                            NaN   
2                  Unchecked                            NaN   
3                  Unchecked                            NaN   
4                        NaN                            NaN   

             Ethnicity  If other ethnicity, please specify  \
0  Non-Hispanic/Latino                                 NaN   
1                  NaN                                 NaN   
2      Hispanic/Latino                                 NaN   
3  Non-Hispanic/Latino                                 NaN   
4                  NaN                                 NaN   

  Country of birth/origin: State of Residence Years in State of Residence  \
0                   NY NY          Manhattan                          1.5   
1                      NaN                NaN                         NaN   
2                     Cuba            Florida                          49   
3                      USA                NJ                          NaN   
4                      NaN                NaN                         NaN   

   Years in USA Complete? Visit Date                            Visit Type  \
0          24.0  Complete 2021-11-17                     Visit 1: Baseline   
1           NaN       NaN 2022-01-06  Visit 3: 60 days (2 months) +/- 10 d   
2          49.0  Complete 2022-01-12                     Visit 1: Baseline   
3          73.0  Complete 2022-01-27                     Visit 1: Baseline   
4           NaN       NaN 2022-03-14  Visit 3: 60 days (2 months) +/- 10 d   

   Patient Global (VAS):  TENDER Joint Count  SWOLLEN joint Count  \
0                    5.0                14.0                 12.0   
1                    8.0                15.0                 13.0   
2                    6.0                14.0                  6.0   
3                    7.0                 4.0                  6.0   
4     

In [19]:
def one_hot_to_label(row):
    for column in row.index:
        if row[column] == 'Checked':
            return column
        else:
            return None

In [29]:
def _next(l, target):
    return next((i for i in l if i.startswith(target)), None)

In [122]:
def medication(row):
    start_column_name = "Are you currently taking any medications for arthritis?"
    start_column_index = list(row.index).index(start_column_name)
    end_column_name = "Complete?"
    end_column_name = next((i for i in list(row[start_column_index:].index) if i.startswith(end_column_name)), None)
    end_column_index = list(row[start_column_index:].index).index(end_column_name)
    end_column_index = start_column_index + end_column_index
    if row[start_column_name] == "Yes":
        j = 0
        for i, column in enumerate(row[start_column_index+1:end_column_index]):
#             print(i)
            if i==j and column == "Yes":
#                 print("Yes:", row.index[start_column_index+1+i])
                j+4
                return row.index[start_column_index+1+i].split('.')[0]
            else:
#                 print("No:", row.index[start_column_index+1+i])
                j+3
    else:
        return None

'Methotrexate'

In [ ]:
def new_smoker(row):
    if row['Currently Smoking'] == "Yes":
        return "current"

In [161]:
df_matched = pd.DataFrame.from_dict(
    {
        "UNMC_id": df['Record ID'],
        "grp": df.apply(lambda row: medication(row), axis=1),
        "init_group": None,
        "CDate": df['Visit Date'],
        "futime": df['Visit Type'],
        
        # Demography
        "age": df['Age (as of today)'],
        "gender": df['Gender'],
        "final_eductiona": None,
        "race_grp": df.iloc[:,5:14].apply(lambda row: one_hot_to_label(row), axis=1),
        "ethnicity": df['Ethnicity'],
        "weight": None,
        "height": None,
        "BMI": None,
        
        # Habit
        "newsmoker": None, # to be added
        "smkyrs": None, # to be added
        "numcigs": None, # to be added
        "drinker": None, # to be added
        "drinksperwk": None, # to be added
        
        # lab test
        "duration_ra": None, # to be added
        "ara_func_class": None, # to be added
        "num_tnf": None, # to be added
        "num_nontnf": None, # to be added 
        "ndmardused": None, # to be added  / DMARD
        "rfstatus_b": None, # to be added
        "rfstatus_impute": None, # this column should be dropped
        "usresultsRF": df['RF value'], # ?? 7777.0 might be NaN
        "ccpstatus_b": None, # to be added
        "ccpstatus_impute": None, # to be added
        "usresultsCCP3": df['CCP value'], # ?? 7777.0 might be NaN
        
        # other disease
        "hxmi": None, # to be added
        "hxunstab_ang": None, # to be added
        "hxchf": None, # to be added
        "hxother_cv": None, # to be added
        "hxtia": None, # to be added
        "hxstroke": None, # to be added
        "hxcopd": None, # to be added
        "hxhtn": None, # to be added
        "hxdiabetes": None, # to be added
        "hxhld": None, # to be added
        "hx_anycancer": None, # to be added
        "ccpstatus_b": None, # to be added
        "ccpstatus_b": None, # to be added
        "ccpstatus_b": None, # to be added
        
        # medication
        "seatedbp1": None, # to be added
        "seatedbp2": None, # to be added
        "pres_mtx": None, # to be added
        "pres_arava": None, # to be added
        "pres_azulfidine": None, # to be added
        "pres_plaquenil": None, # to be added
        "pres_imuran": None, # to be added
        "pres_minocin": None, # to be added
        "pres_pred": None, # to be added
        "nonpresNSAIDs_use": None, # to be added
        "NSAIDs_use": None, # to be added
        "statin_use": None, # to be added
        "ccpstatus_b": None, # to be added
        "ccpstatus_b": None, # to be added
        
        
        # RA
        "tender_jts_28": df['TENDER Joint Count'], # ! we know each choice in 28
        "swollen_jts_28": df['SWOLLEN joint Count'], # ! we know each choice in 28
        "md_global_assess": None,
        "pt_global_assess": df['Patient Global (VAS):'], # ?? value might not match due to VAS 
        "pt_pain": df['2. How much pain have you had because of rheumatoid arthritis over the past week? Please indicate below how severe your pain has been.'], # ??
        "usresultsCRP": df['CRP value'],
        'usresultsIgA': None, # to be added
        'usresultsIgG': None, # to be added
        'usresultsIgM': None, # to be added
        
        # Not in CORRONA
        "usresultsESR": df['ESR value'],
        "DAS28": df['DAS28 Score'], # need to calculate
    }
)
df_matched

,UNMC_id,grp,init_group,CDate,futime,age,gender,final_eductiona,race_grp,ethnicity,weight,height,BMI,newsmoker,smkyrs,numcigs,drinker,drinksperwk,duration_ra,ara_func_class,num_tnf,num_nontnf,ndmardused,rfstatus_b,rfstatus_impute,usresultsRF,ccpstatus_b,ccpstatus_impute,usresultsCCP3,hxmi,hxunstab_ang,hxchf,hxother_cv,hxtia,hxstroke,hxcopd,hxhtn,hxdiabetes,hxhld,hx_anycancer,seatedbp1,seatedbp2,pres_mtx,pres_arava,pres_azulfidine,pres_plaquenil,pres_imuran,pres_minocin,pres_pred,nonpresNSAIDs_use,NSAIDs_use,statin_use,tender_jts_28,swollen_jts_28,md_global_assess,pt_global_assess,pt_pain,usresultsCRP,usresultsIgA,usresultsIgG,usresultsIgM,usresultsESR,DAS28
0,EARA-01,None,None,2021-11-17,Visit 1: Baseline,NaN,Female,None,Race (choice=White/Caucasian),Non-Hispanic/Latino,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,29.0,None,None,134.1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,14.0,12.0,None,5.0,5.0,7.10,None,None,None,20.0,5.86
1,EARA-01,Methotrexate,None,2022-01-06,Visit 3: 60 days (2 months) +/- 10 d,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,7777.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,15.0,13.0,None,8.0,8.0,4.00,None,None,None,27.0,6.54
2,EARA-02,None,None,2022-01-12,Visit 1: Baseline,54.0,Female,None,Race (choice=White/Caucasian),Hispanic/Latino,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,17.0,None,None,87.9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,14.0,6.0,None,6.0,NaN,40.20,None,None,None,62.0,6.51
3,EARA-03,Methotrexate,None,2022-01-27,Visit 1: Baseline,73.0,Male,None,Race (choice=White/Caucasian),Non-Hispanic/Latino,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,15.0,None,None,0.5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4.0,6.0,None,7.0,7.0,16.40,None,None,None,69.0,5.62
4,EARA-03,Methotrexate,None,2022-03-14,Visit 3: 60 days (2 months) +/- 10 d,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7777.0,None,None,7777.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3.0,0.0,None,4.0,5.0,7777.00,None,None,None,25.0,3.78
5,EARA-03,Methotrexate,None,2022-05-19,Visit 4 (optional): 120 days (4 months) +/- 10 d,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7777.0,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,None,6.0,6.0,7777.00,None,None,None,12.0,2.58
6,EARA-04,None,None,2022-02-07,Visit 1: Baseline,58.0,Female,None,Race (choice=White/Caucasian),Non-Hispanic/Latino,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,22.6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,18.0,14.0,None,8.0,2.0,7.80,None,None,None,38.0,5.81
7,EARA-04,Methotrexate,None,2022-03-16,Visit 2: 30 days (1 month) +/- 10 d,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7777.0,None,None,7777.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6.0,2.0,None,7.0,4.0,7777.00,None,None,None,300.0,5.13
8,EARA-04,Methotrexate,None,2022-04-13,Visit 3: 60 days (2 months) +/- 10 d,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7777.0,None,None,7777.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0,None,5.0,3.0,7777.00,None,None

In [148]:
def calculate_DAS28_CRP(row):
    DAS28_CRP = 0.56*math.sqrt(row['TENDER Joint Count']) + 0.28*math.sqrt(
        row['SWOLLEN joint Count']) + 0.014*row['Patient Global (VAS):'] + 0.36*np.log(row['CRP value']+1) + 0.96
    return DAS28_CRP
def calculate_DAS28_ESR(row):
    DAS28_ESR = 0.56*math.sqrt(row['TENDER Joint Count']) + 0.28*math.sqrt(
        row['SWOLLEN joint Count']) + 0.014*row['Patient Global (VAS):'] + 0.70*np.log(row['ESR value'])
    return DAS28_ESR

In [160]:
df.loc[:,'DAS28 Score']

0     5.86
1     6.54
2     6.51
3     5.62
4     3.78
5     2.58
6     5.81
7     5.13
8     3.08
9     2.47
10     NaN
11    4.33
12    3.78
13    4.06
14    6.50
15    3.68
16    2.97
17    6.89
18    5.80
19    4.20
20    3.79
21    2.59
22     NaN
23    4.35
24    3.64
25    5.37
Name: DAS28 Score, dtype: float64

In [158]:
df.apply(lambda row: calculate_DAS28_CRP(row), axis=1)

0     4.848348
1     4.829823
2     5.163823
3     3.892186
4     5.211208
5     4.269260
6     5.278453
7     6.050954
8     4.255260
9     4.199260
10         NaN
11    2.873398
12    5.053260
13    3.031944
14    4.339852
15    2.263305
16    1.418291
17    5.391258
18    4.698063
19    3.006221
20    1.935130
21    5.039260
22         NaN
23    3.447879
24    5.053260
25    4.335240
dtype: float64

In [159]:
df.apply(lambda row: calculate_DAS28_ESR(row), axis=1)

0     5.232289
1     5.597511
2     5.754179
3     4.867732
4     3.279162
5     1.823435
6     6.081853
7     5.858342
8     2.450838
9     2.346543
10         NaN
11    3.450060
12    3.532664
13    3.683734
14    5.649513
15    3.176668
16    2.319615
17    5.818260
18    5.055866
19    3.461256
20    3.659133
21    2.465810
22         NaN
23    3.716596
24    3.271791
25    4.321339
dtype: float64